In [1]:
import pandas as pd
import plotly.graph_objects as go
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path
from pprint import pprint

radial_schema_xlsx = Path(
    "C:/Code/bio_cluster/backend/bio_cluster/src/database/excel/WKO_database_branchen_gesamt.xlsx")

df = pd.read_excel(radial_schema_xlsx)

In [2]:
df_drop = df.drop_duplicates()
df_drop = df_drop.drop_duplicates(subset=['company_name', 'branche_layer_3'])
duplicateRowsDF = df_drop[df_drop.duplicated(['company_name'], keep=False)]
duplicateRowsDF.sort_values('company_name')
duplicateRowsDF.set_index(['company_name', 'branche_layer_3'], inplace=True)

In [3]:
size_factor = 100

layer_3 = df_drop[['branche_layer_1', 'branche_layer_2', 'branche_layer_3']].drop_duplicates(subset=['branche_layer_3'])
dups = df_drop.pivot_table(index = 'branche_layer_3', aggfunc ='size') 
dups = dups.reset_index()
dups.rename(columns={0: "members"}, inplace=True)
layer_3.reset_index(inplace=True)
layer_3['members'] = dups['members']
layer_3.drop(columns=['index'], inplace=True)
layer_3['size'] = (layer_3['members']/(max(layer_3['members']) - min(layer_3['members']))) * size_factor


layer_2 = df_drop[['branche_layer_1', 'branche_layer_2']].drop_duplicates(subset=['branche_layer_2'])
dups = df_drop.pivot_table(index = 'branche_layer_2', aggfunc ='size') 
dups = dups.reset_index()
dups.rename(columns={0: "members"}, inplace=True)
layer_2.reset_index(inplace=True)
layer_2['members'] = dups['members']
layer_2.drop(columns=['index'], inplace=True)
layer_2['size'] =(layer_2['members']/(max(layer_2['members']) - min(layer_3['members']))) * size_factor


layer_1 = df_drop['branche_layer_1'].drop_duplicates()
dups = df_drop.pivot_table(index = 'branche_layer_1', aggfunc ='size') 
dups = dups.reset_index()
dups.rename(columns={0: "members"}, inplace=True)
layer_1 = layer_1.to_frame()
layer_1.reset_index(inplace=True)
layer_1['members'] = dups['members']
layer_1.drop(columns=['index'], inplace=True)
layer_1['size'] =(layer_1['members']/(max(layer_1['members']) - min(layer_3['members']))) * size_factor


In [4]:
edge_li = []
for index, new_df in duplicateRowsDF.groupby(level=0):
    #print("New Group")
    li = []
    for L3, row in new_df.iterrows():
        #print(L3[0], L3[1], len(new_df))
        li.append(L3[1])
    for i in range(0,len(li)):
        A = li[i]
        for j in range(i+1, len(li)):
            B = li[j]
            edge = [A, B]
            edge_li.append(edge)
edge_df = pd.DataFrame(edge_li, columns= ['A','B'])
edge_info = edge_df.groupby(edge_df.columns.tolist(),as_index=False).size()
edge_info = edge_info.reset_index()
edge_info.rename(columns={0: "size"}, inplace=True)
edge_info["weight"] = (edge_info["size"]/(max(edge_info["size"])-min(edge_info["size"])))

dups = df_drop.pivot_table(index = 'branche_layer_3', aggfunc ='size') 
dups = dups.reset_index()
dups.rename(columns={0: "size"}, inplace=True)
dups["weight"] = (dups["size"]/(max(dups["size"])-min(dups["size"])))
print(dups)

                                      branche_layer_3  size    weight
0                                    Agrarunternehmer  1466  0.193327
1                                           Bootbauer   127  0.016748
2      Briefumschlag- und Papierausstattungsindustrie     1  0.000132
3              Büro- und Organisationsmittelindustrie    13  0.001714
4                           Bürsten- und Pinselmacher    17  0.002242
..                                                ...   ...       ...
76                        Wärmeversorgungsunternehmen   290  0.038243
77                         Zigarettenpapierkonfektion     1  0.000132
78                       industrielle Bautischlereien    41  0.005407
79  sonstige Sanitär-, Heizungs- und Lüftungstechn...   196  0.025847
80               sonstige holzverarbeitende Industrie   125  0.016484

[81 rows x 3 columns]


In [5]:
layer_3

,branche_layer_1,branche_layer_2,branche_layer_3,members,size
0,Gewerbe und Handwerk,Gewerbliche Dienstleister (gesamt),Agrarunternehmer,1466,19.332718
1,Gewerbe und Handwerk,Tischler und Holzgestalter (gesamt),Bootbauer,127,1.674799
2,Industrie,Industr. Hersteller von Produkten aus Papier u...,Briefumschlag- und Papierausstattungsindustrie,1,0.013187
3,Industrie,Industr. Hersteller von Produkten aus Papier u...,Büro- und Organisationsmittelindustrie,13,0.171436
4,Gewerbe und Handwerk,Tischler und Holzgestalter (gesamt),Bürsten- und Pinselmacher,17,0.224186
...,...,...,...,...,...
76,Industrie,Industr. Hersteller von Produkten aus Papier u...,Wellpappeindustrie,290,3.824344
77,Gewerbe und Handwerk,Tischler und Holzgestalter (gesamt),Wurzelschnitzer,1,0.013187
78,Gewerbe und Handwerk,Gewerbliche Dienstleister (gesamt),Wärmeversorgung unter 5 km und unter 5 MW,41,0.540683
79,Industrie,Gas- und Wärmeversorgungsunternehmungen (gesamt),Wärmeversorgungsunternehmen,196,2.584729


In [6]:
# edge_info.to_excel(r"G:\_2020\20.013_ka Bioökonomie 2020\3_Bioökonomie-Cluster\Datenbank\Hilfsfiles\branchen_edges.xls", index=False)
# dups.to_excel(r"G:\_2020\20.013_ka Bioökonomie 2020\3_Bioökonomie-Cluster\Datenbank\Hilfsfiles\branchen_nodes.xls", index=False)

In [7]:
##########################  Visualisierung ######################################

In [8]:
list_L1 = layer_1["branche_layer_1"].tolist()
list_L2 = layer_2["branche_layer_2"].tolist()
list_L3 = layer_3["branche_layer_3"].tolist()

L3_L2 = dict(zip(layer_3["branche_layer_3"], layer_3["branche_layer_2"]))
L2_L1 = dict(zip(layer_3["branche_layer_2"], layer_3["branche_layer_1"]))


L3_L2_json = {L2: [node for node in layer_3["branche_layer_3"] if L3_L2[node] == L2]
                    for L2 in list_L2}

json = {}
for L1 in list_L1:
    for L in L3_L2_json:
        if L2_L1[L] == L1:
            json[L1] = {L: L3_L2_json[L] for L in L3_L2_json if L2_L1[L] == L1}


In [9]:
import random

angs_L1 = {}
angs_L2 = {}
angs_L3 = {}
angs_L1_dict = {}
angs_L2_dict = {}
angs_L3_dict = {}

for i, L1 in enumerate(json.keys()):
    angs_L1 = np.linspace(0, 2 * np.pi, 1 + len(json.keys()))
    angs_L1 = np.delete(angs_L1, [0])   
    angs_L1_dict[L1] = angs_L1[i]
    print('L1= ', L1, 'angs= ', angs_L1[i])
    for j, L2 in enumerate(json[L1].keys()):
        angs_L2 = np.linspace(0, 2 * np.pi, 1 + len(json[L1].keys()))
        angs_L2 = np.delete(angs_L2, [0])
        angs_L2_dict[L2] = angs_L2[j]
        print('L2= ', L2, 'angs= ', angs_L2[j])
        for k, L3 in enumerate(json[L1][L2]):
            angs_L3 = np.linspace(0, 2 * np.pi, 1 + len(json[L1][L2]))
            angs_L3 = np.delete(angs_L3, [0])
            angs_L3_dict[L3] = angs_L3[k]
            print('L3= ', L3, 'angs= ', angs_L3[k])
# pprint(angs_L1_dict)
# pprint(angs_L2_dict)
# pprint(angs_L3_dict)

L1=  Gewerbe und Handwerk angs=  2.0943951023931953
L2=  Gewerbliche Dienstleister (gesamt) angs=  1.2566370614359172
L3=  Agrarunternehmer angs=  1.5707963267948966
L3=  Forstunternehmer angs=  3.141592653589793
L3=  Holzzerkleinerer angs=  4.71238898038469
L3=  Wärmeversorgung unter 5 km und unter 5 MW angs=  6.283185307179586
L2=  Tischler und Holzgestalter (gesamt) angs=  2.5132741228718345
L3=  Bootbauer angs=  0.6981317007977318
L3=  Bürsten- und Pinselmacher angs=  1.3962634015954636
L3=  Drechsler angs=  2.0943951023931953
L3=  Hobelwerke angs=  2.792526803190927
L3=  Holzgestalter angs=  3.490658503988659
L3=  Korb- und Möbelflechter angs=  4.1887902047863905
L3=  Parkettbodenleger angs=  4.886921905584122
L3=  Tischler angs=  5.585053606381854
L3=  Wurzelschnitzer angs=  6.283185307179586
L2=  Gärtner und Floristen (gesamt) angs=  3.7699111843077517
L3=  Floristen (Blumenbinder und Blumeneinzelhändler) angs=  1.2566370614359172
L3=  Friedhofsgärtner angs=  2.5132741228718345


In [10]:
np.cos(3)

-0.9899924966004454

G.edges()

In [19]:
def make_edge(x, y, label, width):
    #print('make_edge passed_', label)
    return go.Scatter(x=x,
                      y=y,
                      line=dict(width=width,
                                color='cornflowerblue'),
                      text=label,
                      hoverinfo='text',
                      mode='lines')

G = nx.Graph()
G = nx.from_pandas_edgelist(edge_info, 'A', 'B', ['weight'])
G.add_nodes_from(layer_3['branche_layer_3'])

pos = {}
_pos = dict.fromkeys(angs_L1_dict.keys())

#pos = nx.circular_layout(G)
#nx.draw(G)

r_L1 = 50
r_L2 = 5
r_L3 = 0.5


fig = go.Figure(
    layout=go.Layout(
        title='Cluster-Netzwerk Bioökonomie',
        titlefont_size=16,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=20),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

# print(angs_L1_dict)


for L1 in angs_L1_dict.keys():
    print("cos",np.cos(angs_L1_dict[L1]))
    print("sin",np.sin(angs_L1_dict[L1]))
    _pos[L1] = {"cos": np.cos(angs_L1_dict[L1]),
                "sin": np.sin(angs_L1_dict[L1])}

    # print(_pos[L1])
                         
    # for L2 in angs_L2_dict.keys():

    #     if L3 in pos:
    #         continue
        
    #     else:        
    #         _pos[L2] = np.array([r_L1 * np.cos(angs_L1_dict[L1]) + r_L2 * np.cos(angs_L2_dict[L2]),
    #                              r_L1 * np.sin(angs_L1_dict[L1]) + r_L2 * np.sin(angs_L2_dict[L2])])
    #     for L3 in angs_L3_dict:
    #         if L3 in pos:
    #             #print('Skipped L3', L3)
    #             continue
    #         else:
    #             # print(L3, ' angs = ', angs_L3_dict[L3])
    #             _pos[L3] = np.array([r_L1 * np.cos(angs_L1_dict[L1]) + r_L2 * np.cos(angs_L2_dict[L2]) + r_L3 * np.cos(angs_L3_dict[L3]),
    #                             r_L1 * np.sin(angs_L1_dict[L1]) + r_L2 * np.sin(angs_L2_dict[L2]) + r_L3 * np.sin(angs_L3_dict[L3])])


# for L1 in angs_L1_dict.keys():
#     print("cos",np.cos(angs_L1_dict[L1]))
#     print("sin",np.sin(angs_L1_dict[L1]))
#     pos[L1] = np.array([r_L1 * np.cos(angs_L1_dict[L1]),
#                                  r_L1 * np.sin(angs_L1_dict[L1])])
#     for L2 in angs_L2_dict.keys():

#         if L3 in pos:
#             continue
        
#         else:        
#             pos[L2] = np.array([r_L1 * np.cos(angs_L1_dict[L1]) + r_L2 * np.cos(angs_L2_dict[L2]),
#                                  r_L1 * np.sin(angs_L1_dict[L1]) + r_L2 * np.sin(angs_L2_dict[L2])])
#         for L3 in angs_L3_dict:
#             if L3 in pos:
#                 #print('Skipped L3', L3)
#                 continue
#             else:
#                 # print(L3, ' angs = ', angs_L3_dict[L3])
#                 pos[L3] = np.array([r_L1 * np.cos(angs_L1_dict[L1]) + r_L2 * np.cos(angs_L2_dict[L2]) + r_L3 * np.cos(angs_L3_dict[L3]),
#                                 r_L1 * np.sin(angs_L1_dict[L1]) + r_L2 * np.sin(angs_L2_dict[L2]) + r_L3 * np.sin(angs_L3_dict[L3])])

            # print(L3, pos[L3])
            
#         fig.add_annotation(
#             x=pos[L3][0] - r_L3,
#             y=pos[L3][1] + r_L3 * 2,
#             text=L2)
#     fig.add_annotation(
#         x=pos[L2][0] - r_L1,
#         y=pos[L2][1] + r_L1 * 2,
#         text=L2)

# fig.add_annotation(
#     x=pos[L1][0] - r_L1,
#     y=pos[L1][1] + r_L1 * 2,
#     text=L1)

# fig.update_annotations(
#         dict(
#             xref="x",
#             yref="y",
#             font_size=15,
#             bgcolor ='honeydew',
#             showarrow=False,
#             arrowhead=7,
#             ax=0,
#             ay=-40
#             ))

# edge_x = []
# edge_y = []
# edge_trace = []

# for edge in G.edges():
#     # print(edge, pos[edge[0]][0], pos[edge[0]][1], G.edges()[edge]['weight'])
#     x0, y0 = pos[edge[0]][0], pos[edge[0]][1]
#     x1, y1 = pos[edge[1]][0], pos[edge[1]][1]
#     edge_x.append(x0)
#     edge_x.append(x1)
#     edge_x.append(None)
#     edge_y.append(y0)
#     edge_y.append(y1)
#     edge_y.append(None)
#     label = G.edges()[edge]['weight']

#     trace = make_edge([x0, x1, None], [y0, y1, None], label=label, width=20*G.edges()[edge]['weight'])

#     edge_trace.append(trace)
    
    
# node_x = []
# node_y = []
# labels_node = []
# for node in G.nodes():
#     #print(pos[node])
#     x, y = pos[node]
#     node_x.append(x)
#     node_y.append(y)
#     labels_node.append(node)
    

# node_trace = go.Scatter(
#     x=node_x, y=node_y,
#     text=labels_node,
#     mode='markers+text',
#     hoverinfo='text',
#     textposition='bottom center',
#     marker=dict(
#         showscale=True,
#         size=dups['weight']*100,
#         # colorscale options
#         # 'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
#         # 'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
#         # 'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
#         colorscale='Greens',
#         reversescale=True,
#         color=[],
#         colorbar=dict(
#             thickness=1,
#             title='Anzahl an Verbindungen',
#             xanchor='left',
#             titleside='right'
#         ),
#         line_width=2))

# node_adjacencies = []
# node_text = []
# for node, adjacencies in enumerate(G.adjacency()):
#     node_adjacencies.append(len(adjacencies[1]))
# node_trace.marker.color = node_adjacencies


# for trace in edge_trace:
#     fig.add_trace(trace)
# fig.add_trace(node_trace)
# fig.show()

cos -0.4999999999999998
sin 0.8660254037844387
cos -0.5000000000000004
sin -0.8660254037844385
cos 1.0
sin -2.4492935982947064e-16
